In [19]:
import pandas as pd
import numpy as np
import re
import os
import datetime
from collections import defaultdict
from pathlib import Path

import json
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, ConceptsOptions, EmotionOptions, EntitiesOptions, KeywordsOptions, SemanticRolesOptions, SentimentOptions, CategoriesOptions, SyntaxOptions, SyntaxOptionsTokens
authenticator = IAMAuthenticator( apikey )
nlu = NaturalLanguageUnderstandingV1( version='2018-11-16', authenticator=authenticator )
nlu.set_service_url( url )

In [11]:
apikey = "CnELF_rkq2iTbjiq07Q0FdE2O48HhDn-UsgTTIrP1rK3"  # <-- PASTE YOUR APIKEY HERE
url = "https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/88fb2036-5c04-4fba-80a5-9126387d5c89"  # <-- PASTE YOUR SERVICE URL HERE

In [30]:
def watson_analyze(message):

    """
    Extract keywords and sematic roles from text
    Input : message
    Output : Array of action verbs, Array of Keywords
    """
    try:
        result = nlu.analyze(text=message,features=Features(emotion=EmotionOptions() ,sentiment=SentimentOptions())).get_result()
    except:
        return pd.Series([0,0,0,0,0,0])
    try:
        sad_score = result["emotion"]["document"]["emotion"]["sadness"]
    except: 
        sad_score = 0
    try:
        joy_score = result["emotion"]["document"]["emotion"]["joy"]
    except: 
        joy_score = 0
    try:
        fear_score = result["emotion"]["document"]["emotion"]["fear"]
    except: 
        fear_score = 0
    try:
        disgust_score = result["emotion"]["document"]["emotion"]["disgust"]
    except: 
        disgust_score = 0
    try:
        anger_score = result["emotion"]["document"]["emotion"]["anger"]
    except: 
        anger_score = 0
    try:
        semtiment_score = result["sentiment"]["document"]["emotion"]["score"]
    except: 
        semtiment_score = 0

    return pd.Series([sad_score, joy_score,fear_score,disgust_score,anger_score,semtiment_score])

In [3]:
df = pd.read_csv('../data/evaluation_volunteer_text_1000.csv')

In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
df.head()

,text,sad-watson,joy-watson,fear-watson,disgust-watson,anger-watson,sentiment-watson
0,u still there,NaN,NaN,NaN,NaN,NaN,NaN
1,call my #,NaN,NaN,NaN,NaN,NaN,NaN
2,"Really, you are better than this (calling thes...",NaN,NaN,NaN,NaN,NaN,NaN
3,We're a bunch of guys working to reduce demand...,NaN,NaN,NaN,NaN,NaN,NaN
4,"lots of danger, including jail",NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df["sad-watson"] = np.nan
df["joy-watson"] = np.nan
df["fear-watson"] = np.nan
df["disgust-watson"] = np.nan
df["anger-watson"] = np.nan
df["sentiment-watson"] = np.nan

In [ ]:
df[["sad-watson", "joy-watson","fear-watson" ,"disgust-watson", "anger-watson", "sentiment-watson"]] = df["text"].apply(
    lambda x: watson_analyze(x)
)

In [ ]:
df.to_csv("../data/watson_label.csv")